In [1]:
#librerias
#%pip install -r requirements.txt
!python3 pip install -r requirements.txt

python3: can't open file 'c:\\Users\\chara\\Desktop\\Rapido-Furioso\\pip': [Errno 2] No such file or directory


In [2]:
import pandas as pd
from connection_script import engine_bodega, engine_etl

EXTRAER DATOS:
se extrae los las columnas neecesarias desde la tabla clientes_mensajeroaquitoy de la base de datos, y estos se cargan en un Dataframe

In [3]:
query = """
   SELECT 
    c.id,
    c.activo,
    c.telefono,
    CONCAT(a.first_name, ' ', a.last_name) AS nombre_mensajero
FROM 
    clientes_mensajeroaquitoy c
LEFT JOIN 
    auth_user a ON c.id = a.id;


"""

# Ejecutar la consulta y cargar los datos en un DataFrame de Pandas
mensajeros_df = pd.read_sql(query, engine_bodega)


Transformacion

Cambios del nombre de la columna

In [4]:
#transformar datos
mensajeros_df["id_mensajero"] = mensajeros_df["id"]
mensajeros_df["telefono_mensajero"] = mensajeros_df["telefono"]

mensajeros_df.drop(columns=["id", "telefono"], inplace=True)


In [5]:
#cargar para la base de datos
# Verificar si ya existen registros con 'id_estado' en la base de datos ETL para evitar duplicados
existing_ids_query = "SELECT id_mensajero FROM mensajero"
existing_ids = pd.read_sql(existing_ids_query, engine_etl)

# Filtrar los registros que ya existen en la base de datos ETL
mensajeros_df = mensajeros_df[~mensajeros_df['id_mensajero'].isin(existing_ids['id_mensajero'])]

# Cargar los datos transformados en la base de datos ETL
mensajeros_df.to_sql('mensajero', con=engine_etl, if_exists="append", index=False)

0